In [62]:
%run InitialSetup.ipynb

In [ ]:
"1) Quick Calibration of parameters using Hagan"
#Vectorize functions
H_Shift = np.vectorize(sigma_shifted_CEV_Hagan)
Black = np.vectorize(payer_Black)

#payer_Black(N, S0, expiry, tenor, K, sigma)

# Select 1*2 swaption 
test_calib_data = vol_data['1Y 2Y']/10000

S0 = 0.025
K = 0.025
N =  1e+5

# Find all evaluation dates
eval_dates = [DateParser.parseFormatted(datetime.strftime(date,'%Y-%m-%d'), '%Y-%m-%d') for date in test_calib_data.index]
eval_dates = np.array(eval_dates)

# Find all expiries & tenors
expiry =  eval_dates + Period(2, Years)
tenor = expiry + Period(1, Years)

# Define the objective function
opt_func = lambda test: np.sum(np.square(
                    Black(N, S0,expiry, tenor, K, 
                          H_Shift(S0, K, test[0], test[1], test[2], eval_dates, expiry))-
                    Black(N, S0,expiry, tenor, K, test_calib_data)))

# Least Square Minimization - Derivative of Levenberg-Marquardt
'''Note:
test[0] = sigma (Volatility Parameter)
test[1] = delta (Shift Parameter)
test[2] = eta   (Power Parameter)'''
value_LS = optimize.least_squares(opt_func, 
                       x0 = [0, 0, 0.5],
                       ftol = 1e-15, 
                       xtol=1e-15, 
                       gtol=1e-09)
value_LS

In [ ]:
"2) Diffusion of the Forward Rates"
# Initial Parameters
F0 = 0
sigma_1 = 0.001
sigma_2 = 0.002
rho = 0.002
beta = 0.5
timestep = 1/52 #Weekly timestep

# Iteration

    